# 🔧 Soluciones Corregidas - Lab 2

## ⚠️ Problema Identificado

El error ocurre porque `@function_tool` no soporta tipos complejos como `List[Dict]`.

### Solución:
1. Usar tipos simples (str, int, float, bool)
2. Pasar datos complejos como JSON strings
3. Crear modelos Pydantic para tipos complejos

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict, List, Optional
from pydantic import BaseModel
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import json
from datetime import datetime

load_dotenv(override=True)

True

---
## ✅ Solución 1: Usar JSON Strings

In [5]:
# Base de datos de contactos
contacts_database = [
    {
        "name": "Alice Johnson",
        "email": "enriquedlrm98@usal.es",
        "company": "TechCorp",
        "role": "CEO",
        "industry": "SaaS"
    },
    {
        "name": "Bob Smith",
        "email": "enriquedlrm98@gmail.com",
        "company": "FinanceCo",
        "role": "CTO",
        "industry": "Finance"
    }
]

@function_tool
def get_contacts(industry: str = "") -> str:
    """
    Retrieve contact list from database, optionally filtered by industry.
    Returns a JSON string of contacts.
    
    Args:
        industry: Optional industry filter (e.g., 'SaaS', 'Finance', 'Healthcare'). Leave empty for all contacts.
    
    Returns:
        JSON string containing list of contacts with name, email, company, role, and industry
    """
    if industry:
        filtered = [c for c in contacts_database if c["industry"].lower() == industry.lower()]
        return json.dumps(filtered, indent=2)
    return json.dumps(contacts_database, indent=2)


@function_tool
def send_mail_merge(subject: str, html_body: str, contacts_json: str) -> str:
    """
    Send personalized emails to multiple contacts using mail merge.
    
    The html_body can include placeholders like {{name}}, {{company}}, {{role}}
    which will be replaced with actual contact data.
    
    Args:
        subject: Email subject line
        html_body: HTML email body with placeholders like {{name}}, {{company}}, {{role}}
        contacts_json: JSON string containing list of contacts (use get_contacts to retrieve)
    
    Returns:
        JSON string with send status and details
    """
    try:
        contact_list = json.loads(contacts_json)
    except json.JSONDecodeError:
        return json.dumps({"error": "Invalid contacts JSON"})
    
    sent_count = 0
    failed = []
    
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("enriquedlrm98@gmail.com")  # Change to your verified sender
    
    for contact in contact_list:
        try:
            # Personalizar el contenido
            personalized_body = html_body
            for key, value in contact.items():
                placeholder = f"{{{{{key}}}}}"
                personalized_body = personalized_body.replace(placeholder, str(value))
            
            # Enviar email
            to_email = To(contact["email"])
            content = Content("text/html", personalized_body)
            mail = Mail(from_email, to_email, subject, content).get()
            
            response = sg.client.mail.send.post(request_body=mail)
            
            if response.status_code == 202:
                sent_count += 1
            else:
                failed.append(contact["email"])
                
        except Exception as e:
            failed.append({"email": contact["email"], "error": str(e)})
    
    result = {
        "status": "completed",
        "sent": sent_count,
        "failed": len(failed),
        "total": len(contact_list),
        "failed_contacts": failed
    }
    
    return json.dumps(result, indent=2)

# Test
print("✅ Tools creadas correctamente")


✅ Tools creadas correctamente


---
## ✅ Solución 2: Analytics Tools (Simplificadas)

In [6]:
# Base de datos de analytics
email_analytics = []

@function_tool
def log_email_sent(recipient: str, subject: str, campaign: str) -> str:
    """
    Log that an email was sent for analytics tracking.
    
    Args:
        recipient: Email address of recipient
        subject: Subject line of the email
        campaign: Campaign name or identifier (e.g., 'SOC2_Healthcare_Feb2025')
    
    Returns:
        JSON string with confirmation and tracking ID
    """
    tracking_id = f"EMAIL_{len(email_analytics) + 1}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    log_entry = {
        "tracking_id": tracking_id,
        "recipient": recipient,
        "subject": subject,
        "campaign": campaign,
        "sent_at": datetime.now().isoformat(),
        "opened": False,
        "clicked": False,
        "replied": False
    }
    
    email_analytics.append(log_entry)
    
    return json.dumps({
        "status": "logged",
        "tracking_id": tracking_id
    })


@function_tool
def get_campaign_stats(campaign: str) -> str:
    """
    Get statistics for a specific email campaign.
    
    Args:
        campaign: Campaign name to analyze
    
    Returns:
        JSON string with statistics including sent count, open rate, click rate, reply rate
    """
    campaign_emails = [e for e in email_analytics if e["campaign"] == campaign]
    
    if not campaign_emails:
        return json.dumps({"error": f"No emails found for campaign '{campaign}'"})
    
    total = len(campaign_emails)
    opened = sum(1 for e in campaign_emails if e["opened"])
    clicked = sum(1 for e in campaign_emails if e["clicked"])
    replied = sum(1 for e in campaign_emails if e["replied"])
    
    stats = {
        "campaign": campaign,
        "total_sent": total,
        "opens": opened,
        "clicks": clicked,
        "replies": replied,
        "open_rate": f"{(opened/total*100):.1f}%" if total > 0 else "0%",
        "click_rate": f"{(clicked/total*100):.1f}%" if total > 0 else "0%",
        "reply_rate": f"{(replied/total*100):.1f}%" if total > 0 else "0%"
    }
    
    return json.dumps(stats, indent=2)

print("✅ Analytics tools creadas")

✅ Analytics tools creadas


---
## ✅ Solución 3: Agentes Especializados

In [7]:
# Agentes de ventas originales
instructions1 = """You are a sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write professional, serious cold emails."""

instructions2 = """You are a humorous, engaging sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write witty, engaging cold emails that are likely to get a response."""

instructions3 = """You are a busy sales agent working for ComplAI, 
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write concise, to the point cold emails."""

sales_agent1 = Agent(name="Professional Sales Agent", instructions=instructions1, model="gpt-4o-mini")
sales_agent2 = Agent(name="Engaging Sales Agent", instructions=instructions2, model="gpt-4o-mini")
sales_agent3 = Agent(name="Concise Sales Agent", instructions=instructions3, model="gpt-4o-mini")

# Convertir a tools
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a professional cold sales email")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write an engaging cold sales email")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a concise cold sales email")

print("✅ Sales agents creados")

✅ Sales agents creados


---
## ✅ Agente de Personalización

In [8]:
personalization_instructions = """
You are a personalization specialist for cold sales emails.

Given:
- A generic email template
- Contact information in JSON format

Your job is to:
1. Add personalized touches that reference their specific industry
2. Include relevant pain points for their role
3. Use placeholders {{name}}, {{company}}, {{role}} for mail merge
4. Keep the core message but make it feel personally written
5. Return the email in HTML format

IMPORTANT: Always include placeholders like {{name}}, {{company}}, {{role}} 
so the email can be personalized for each recipient.
"""

personalization_agent = Agent(
    name="Personalization Specialist",
    instructions=personalization_instructions,
    model="gpt-4o-mini"
)

personalization_tool = personalization_agent.as_tool(
    tool_name="personalize_email",
    tool_description="Personalize a generic email template with placeholders for mail merge"
)

print("✅ Personalization agent creado")

✅ Personalization agent creado


---
## ✅ Agente de Follow-up

In [9]:
followup_instructions = """
You are a follow-up email specialist.

Given the original email that was sent, write a polite, professional follow-up email.

Guidelines:
- Reference the previous email naturally
- Don't be pushy or aggressive
- Provide additional value (new insight, case study, etc.)
- Keep it shorter than the original
- Include a clear, easy call-to-action

Tone based on timing:
- First follow-up (3-5 days): Gentle reminder with added value
- Second follow-up (7-10 days): More direct, highlight urgency
- Third follow-up (14+ days): Break-up email ("Is now not a good time?")
"""

followup_agent = Agent(
    name="Follow-up Specialist",
    instructions=followup_instructions,
    model="gpt-4o-mini"
)

followup_tool = followup_agent.as_tool(
    tool_name="generate_followup",
    tool_description="Generate a follow-up email based on the original email and days since sent"
)

print("✅ Follow-up agent creado")

✅ Follow-up agent creado


---
## 🎯 Sales Manager V2 - Configuración Completa

In [10]:
# Compilar todas las herramientas
advanced_tools = [
    tool1,                    # Professional sales agent
    tool2,                    # Engaging sales agent
    tool3,                    # Concise sales agent
    get_contacts,             # Get contact list
    personalization_tool,     # Personalize emails
    send_mail_merge,          # Send bulk emails
    log_email_sent,           # Track emails
    get_campaign_stats,       # Get analytics
    followup_tool             # Generate follow-ups
]

advanced_sales_manager_instructions = """
You are an Advanced Sales Manager at ComplAI with sophisticated campaign capabilities.

Your workflow for launching a campaign:

1. **Get Contacts**: 
   - Use get_contacts tool to retrieve prospect list
   - Can filter by industry if specified
   - Returns JSON string of contacts

2. **Generate Email**: 
   - Use one of the sales_agent tools (sales_agent1, sales_agent2, or sales_agent3)
   - Choose based on target audience

3. **Personalize**: 
   - Use personalize_email tool to adapt the email
   - Make sure it includes {{name}}, {{company}}, {{role}} placeholders

4. **Send Campaign**: 
   - Use send_mail_merge with:
     - Subject line
     - Personalized HTML body (from step 3)
     - Contacts JSON (from step 1)

5. **Track**: 
   - Use log_email_sent for each recipient to enable analytics

Additional capabilities:
- Query campaign statistics with get_campaign_stats
- Generate follow-ups with generate_followup tool

IMPORTANT: Always use the tools in order. Don't skip steps.
Campaign names should be clear (e.g., "SOC2_Healthcare_Feb2025")
"""

advanced_sales_manager = Agent(
    name="Advanced Sales Manager",
    instructions=advanced_sales_manager_instructions,
    tools=advanced_tools,
    model="gpt-4o-mini"
)

print("✅ Advanced Sales Manager creado con", len(advanced_tools), "herramientas")

✅ Advanced Sales Manager creado con 9 herramientas


---
## 🚀 Test 1: Campaña Básica

In [ ]:
message = """
Launch a simple email campaign:

1. Get all contacts from the SaaS industry
2. Use the engaging sales agent to write an email about SOC2 compliance
3. Personalize it for the SaaS industry
4. Send to all SaaS contacts
5. Track each email with campaign name 'SOC2_SaaS_Test'
"""

with trace("SaaS Campaign"):
    result = await Runner.run(advanced_sales_manager, message)

print("\n" + "="*60)
print("📊 Campaign Result:")
print("="*60)
print(result.final_output)


📊 Campaign Result:
Your email campaign has been successfully launched! Here’s a summary of the steps taken:

1. **Contacts Retrieved**: 1 contact from the SaaS industry.
2. **Email Generated**: Engaging email created about SOC2 compliance.
3. **Personalization**: Email personalized for Alice Johnson at TechCorp.
4. **Emails Sent**: 1 email sent successfully.
5. **Tracking**: Email logged for campaign tracking under the name **SOC2_SaaS_Test**.

If you need to analyze the campaign's effectiveness or have any follow-up actions, feel free to ask!


In [17]:
message = """
Launch a simple email campaign:

1. Get all contacts from the Finance industry
2. Use the engaging sales agent to write an email about SOC2 compliance
3. Personalize it for the Finance industry
4. Send to all Finance contacts
5. Track each email with campaign name 'SOC2_Finance_Test'
"""

with trace("Finance Campaign"):
    result = await Runner.run(advanced_sales_manager, message)

print("\n" + "="*60)
print("📊 Campaign Result:")
print("="*60)
print(result.final_output)



📊 Campaign Result:
The email campaign has been successfully launched! Here’s a summary of the actions taken:

1. **Contacts Retrieved**: 1 contact from the Finance industry.
2. **Email Content**: An engaging email about SOC2 compliance was created.
3. **Email Personalized**: Customized for the recipient, Bob Smith from FinanceCo.
4. **Campaign Sent**: The email was sent to Bob Smith successfully.
5. **Tracking Logged**: The email was tracked under the campaign name 'SOC2_Finance_Test' with tracking ID: `EMAIL_4_20260202110743`.

If you need further actions or follow-ups, just let me know!
